In [1]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import re
import jieba
import os
import pickle
import pandas as pd

d:\anaconda3\envs\aicourse\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
d:\anaconda3\envs\aicourse\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def cut(string): return ' '.join(jieba.cut(string))

In [3]:
DATA_PATH = 'd:/senior/aiCourse/dataSource/'
MODEL_PATH = os.path.join(DATA_PATH,'w2vmodel/wiki_news_cutting.model')
TRAIN_DATA_PATH = os.path.join(DATA_PATH,'wiki_news_cutting.txt')

我在代码外将wiki和news的语料库合并成了一个wiki_news_record.txt的文件，直接作为词向量的训练库输入。

In [4]:
def load_model():
    if os.path.exists(MODEL_PATH):
        print("Model found ! Now loading...")
        return Word2Vec.load(MODEL_PATH)
    else:
        print("Model not found, start training, please wait for about 15 minutes.")
        model= Word2Vec( LineSentence(open(TRAIN_DATA_PATH,encoding='gb18030')), size=100,min_count=1, workers=3)
        model.save(MODEL_PATH)
        return model
#下面直接加载与训练模型

In [5]:
all_word2vec = load_model()

Model found ! Now loading...


In [6]:
from collections import defaultdict
from functools import lru_cache
import datetime

@lru_cache(maxsize=10*100*100)
def find_most_similar(node, topn_in ,model):
    new_expanding_temp = [w for w,s in model.most_similar(node,topn=topn_in)]
    return new_expanding_temp

def get_related_words(initial_words, model):
    """
    @initial_words are initial words we already know
    @model is the word2vec model
    """
    starttime = datetime.datetime.now()
    
    unseen = initial_words
    
    seen = defaultdict(int)
    
    max_size = 1000  # could be greater
    
    while unseen and len(seen) < max_size:
        #if len(seen) % 50 == 0: 
        #    print('seen length : {}'.format(len(seen)))
          
        node = unseen.pop(0)
                
        if seen[node]:
            if seen[node]>10:
                seen[node] += seen[node]/50
            else:
                seen[node]+=1
            new_expanding =  find_most_similar(node, 10 ,model)
            unseen+=new_expanding
            continue
               
        new_expanding =  find_most_similar(node, 10 ,model)  
        unseen += new_expanding
        seen[node] += 1
        
        # optimal: 1. score function could be revised
        # optimal: 2. using dymanic programming to reduce computing time
    endtime = datetime.datetime.now()
    print('Using : {} s'.format((endtime-starttime).seconds))
    
    return seen

In [7]:
def load_say_word():
    say_word_file= os.path.join(DATA_PATH,'say.pickle')
    if os.path.exists(say_word_file):  
        print('Pickle file found ! Now loading!')
        result = pickle.load(open(say_word_file,'rb'))
    else:
        print('Pickle file not found ! Starting searching, please wait for about 2 minutes.')
        related_words = get_related_words(['说', '表示'], all_word2vec)
        pickle.dump(related_words,open(say_word_file,'wb'),pickle.HIGHEST_PROTOCOL)
        result = pickle.load(open(say_word_file,'rb'))

    return result

In [8]:
say_word=load_say_word()

Pickle file found ! Now loading!


# 实体识别

In [9]:
text1 = """新华社华盛顿4月26日电 美国总统特朗普26日表示，美国将撤销在《武器贸易条约》上的签字。

特朗普当天在美国印第安纳州首府印第安纳波利斯举行的美国全国步枪协会年会上说，《武器贸易条约》是一个“严重误导的条约”，美国将撤销在该条约上的签字，联合国将很快收到美国正式拒绝该条约的通知。"""

In [10]:
text = """王近亲表示他今晚吃了十个包子"""

In [11]:
text2 = """昨日，雷先生说，交警部门罚了他 16 次，他只认了一次，交了一次罚款，拿到法院的判决书后，会前往交警队，要求撤销此前的处罚。

律师：不依法粘贴告知单有谋取罚款之嫌。
陕西金镝律师事务所律师骆裕德说，这起案件中，交警部门在处理交通违法的程序上存在问题。司机违停了，交警应将处罚单张贴在车上，并告知不服可以行使申请
复议和提起诉讼的权利。这既是交警的告知义务，也是司机的知情权利。交警如果这么做了，本案司机何以被短时间内处罚 16 次后才知晓被罚？程序违法，为罚而罚，没
有起到教育的目的。"""

In [12]:
text3= """中新网6月23日电 (记者潘旭临) 意大利航空首席商务官乔治先生22日在北京接受中新网记者专访时表示，意航确信中国市场对意航的重要性，目前意航已将发展中国市场提升到战略层级的高度，
未来，意航将加大在华布局，提升业务水平。
到意大利航空履职仅7个月的乔治，主要负责包括中国市场在内的亚太业务。
乔治称，随着对华业务不断提升，意航明年可能会将每周4班提高到每天一班。同时，意航会借罗马新航站楼启用之际，吸引更多中国旅客到意大利旅游和转机。
此外，还将加大对北京直飞航线的投资，如翻新航班座椅，增加电视中有关中国内容的娱乐节目、提高机上中文服务、餐饮服务、完善意航中文官方网站，提升商务舱和普通舱的舒适度等。"""

In [13]:
symbol='[^(（|）|《|》|“|”|「|」|、|\-|：|·|；|?|,|.|:|"|\'|\')]'
#print(symbol)
def token(string):
    return ''.join(re.findall(symbol,string))

cut(token(text1))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\dell\AppData\Local\Temp\jieba.cache
Loading model cost 0.801 seconds.
Prefix dict has been built succesfully.


'新华社 华盛顿 4 月 26 日电   美国 总统 特朗普 26 日 表示 ， 美国 将 撤销 在 武器 贸易 条约 上 的 签字 。 \n \n 特朗普 当天 在 美国 印第安纳州 首府 印第安纳波利斯 举行 的 美国 全国 步枪 协会 年 会上 说 ， 武器 贸易 条约 是 一个 严重 误导 的 条约 ， 美国 将 撤销 在 该 条约 上 的 签字 ， 联合国 将 很快 收到 美国 正式 拒绝 该 条约 的 通知 。'

In [14]:
import os
from pyltp import Segmentor
from pyltp import Postagger
from pyltp import NamedEntityRecognizer
from pyltp import Parser
from pyltp import SementicRoleLabeller

LTP_DATA_DIR = 'D://senior/aiCourse/dataSource/ltp_data_v3.4.0/'  # ltp模型目录的路径
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')
pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')
ner_model_path = os.path.join(LTP_DATA_DIR, 'ner.model')
par_model_path = os.path.join(LTP_DATA_DIR, 'parser.model')
srl_model_path = os.path.join(LTP_DATA_DIR, 'pisrl_win.model')

In [15]:
def pyltp_cutting(sentence):
    segmentor = Segmentor()  # 初始化实例
    segmentor.load(cws_model_path)  # 加载模型
    result = segmentor.segment(sentence)  # 分词
    #print ('\t'.join(words))
    segmentor.release()  # 释放模型
    return result

In [16]:
def pyltp_postagger(words):
    postagger = Postagger() # 初始化实例
    postagger.load(pos_model_path)  # 加载模型

    result = postagger.postag(words)  # 词性标注

   # print ('\t'.join(postags))
    postagger.release()  # 释放模型
    return result

In [17]:
def pyltp_ner(words,postags):
    recognizer = NamedEntityRecognizer() # 初始化实例
    recognizer.load(ner_model_path)  # 加载模型


    result = recognizer.recognize(words, postags)  # 命名实体识别

    #print ('\t'.join(netags))
    recognizer.release()  # 释放模型
    return result

In [18]:
def pyltp_parser(words,postags):
    parser = Parser() # 初始化实例
    parser.load(par_model_path)  # 加载模型
    result = parser.parse(words, postags)  # 句法分析

    #print ("\t".join("%d:%s" % (arc.head, arc.relation) for arc in arcs))
    parser.release()  # 释放模型
    return result

In [19]:
def pyltp_role_parsring(words, postags, arcs):
    labeller = SementicRoleLabeller() # 初始化实例
    labeller.load(srl_model_path)  # 加载模型
    result = labeller.label(words, postags, arcs)  # 语义角色标注
    #for role in roles:
    #    print (words[role.index], "".join(
    #        ["%s:(%d,%d)" % (arg.name, arg.range.start, arg.range.end) for arg in role.arguments]))
    labeller.release()  # 释放模型
    return result

In [20]:
words=pyltp_cutting(cut(token(text3)))

postags=pyltp_postagger(words)
netags = pyltp_ner(words,postags)
arcs = pyltp_parser(words,postags)
roles = pyltp_role_parsring(words, postags, arcs)

In [21]:

for role in roles:
    if words[role.index] in say_word.keys():
        if say_word[words[role.index]]>5:
            print (words[role.index], "".join(
        ["%s:(%d,%d)" % (arg.name, arg.range.start, arg.range.end) for arg in role.arguments]))

表示 A0:(4,11)TMP:(12,19)A1:(22,57)
确信 A0:(22,22)A1:(24,29)


In [22]:
people = []
verb=[]
contents=[]
for role in roles:
    if words[role.index] in say_word.keys():
        if say_word[words[role.index]]>5:
            parsing_argument = []
            for arg in role.arguments:
                parsing_argument.append(arg.name)
            if 'A0' in parsing_argument and 'A1' in parsing_argument:
                verb.append ( words[role.index])
                for arg in role.arguments:
                    if arg.name =='A0':
                        people.append(''.join(words[i] for i in range(arg.range.start, arg.range.end+1)))
                    if arg.name =='A1':
                        contents.append(''.join(words[i] for i in range(arg.range.start, arg.range.end+1)))



In [23]:
from tabulate import tabulate

table = []
for i in range(len(people)):
    table.append([people[i],verb[i],contents[i]])
table

[['记者潘旭临意大利航空首席商务官乔治先生',
  '表示',
  '意航确信中国市场对意航的重要性，目前意航已将发展中国市场提升到战略层级的高度，\n未来，意航将加大在华布局，提升业务水平'],
 ['意航', '确信', '中国市场对意航的重要性']]

In [29]:
length = len(words)
for i in range(length):
    print('{} {} {} '.format(words[i],postags[i],netags[i]))
    #if postags[i]=='nh' or postags[i]=='nz':
    #if netags[i]!='O':
     #   print('{} {} {} '.format(words[i],postags[i],netags[i]))

中新网 n O 
6月 nt O 
23日 nt O 
电 n O 
记者 n O 
潘旭临 nh S-Nh 
意大利 ns S-Ns 
航空 n O 
首席 n O 
商务官 n O 
乔治 nh S-Nh 
先生 n O 
22日 nt O 
在 p O 
北京 ns S-Ns 
接受 v O 
中新网 nz O 
记者 n O 
专访 v O 
时 n O 
表示 v O 
， wp O 
意航 j O 
确信 v O 
中国 ns S-Ns 
市场 n O 
对 p O 
意航 j O 
的 u O 
重要性 n O 
， wp O 
目前 nt O 
意航 j O 
已 d O 
将 p O 
发展 v O 
中国 ns S-Ns 
市场 n O 
提升 v O 
到 v O 
战略 n O 
层级 n O 
的 u O 
高度 n O 
， wp O 

 v O 
未来 nt O 
， wp O 
意航 j O 
将 d O 
加大 v O 
在 p O 
华 j O 
布局 n O 
， wp O 
提升 v O 
业务 n O 
水平 n O 
。 wp O 

到 v O 
意大利 ns S-Ns 
航空 n O 
履职 n O 
仅 d O 
7 m O 
个 q O 
月 n O 
的 u O 
乔治 nh S-Nh 
， wp O 
主要 d O 
负责 v O 
包括 v O 
中国 ns S-Ns 
市场 n O 
在内 u O 
的 u O 
亚 j O 
太 j O 
业务 n O 
。 wp O 

 n O 
乔治 nh S-Nh 
称 v O 
， wp O 
随着 p O 
对 p O 
华 j O 
业务 n O 
不断 d O 
提升 v O 
， wp O 
意航 j O 
明年 nt O 
可能 v O 
会 v O 
将 p O 
每周 r O 
4 m O 
班 n O 
提高 v O 
到 v O 
每天 r O 
一 m O 
班 q O 
。 wp O 
同时 n O 
， wp O 
意航 j O 
会 v O 
借 v O 
罗马 ns B-Ni 
新航 j E-Ni 
站楼 n O 
启用 v O 
之际 nd O 
， wp O 
吸引 v O 
更 d O 
多 a O 
中国 ns S-Ns 


In [30]:
cut(token(text1))

'新华社 华盛顿 4 月 26 日电   美国 总统 特朗普 26 日 表示 ， 美国 将 撤销 在 武器 贸易 条约 上 的 签字 。 \n \n 特朗普 当天 在 美国 印第安纳州 首府 印第安纳波利斯 举行 的 美国 全国 步枪 协会 年 会上 说 ， 武器 贸易 条约 是 一个 严重 误导 的 条约 ， 美国 将 撤销 在 该 条约 上 的 签字 ， 联合国 将 很快 收到 美国 正式 拒绝 该 条约 的 通知 。'

In [32]:
def document_frequency(word): 
    return sum(1 for n in news_content if word in n)

def idf(word):
    """Gets the inversed document frequency"""
    return math.log10(len(news_content) / document_frequency(word))

def tf(word, document):
    """
    Gets the term frequemcy of a @word in a @document.
    """
    document=cut(token(document))
    words = document.split()
    
    return sum(1 for w in words if w == word)

In [34]:
tf('意大利',text3)

3